In [23]:
# Random Forest + Bipolar L-Fuzzy PROMETHEE Combined Model
# Step 1: Random Forest Model
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Example historical data of batteries with normalized values
past_data = {
    'Energy Efficiency (Positive)': [0.857, 0.429, 0.8, 0.8],   
    'Cycling Capability (Positive)': [0.091, 0.008, 0.025, 0.091],
    'Lifespan (Positive)': [0.6, 0.2, 0.4, 0.7],
    'Self-Discharge (Positive)': [0.246, 0.005, 0.246, 0.3],
}

# Convert the data into a pandas DataFrame
past_df = pd.DataFrame(past_data)

# Historical performance scores 
overall_performance = [0.51735, 0.20275, 0.43420, 0.53320]

# Create and train the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(past_df, overall_performance)

# Get the feature importances (weights for each criterion)
feature_importances = model.feature_importances_

weights = feature_importances

# Save the learned weights
np.save("rf_weights.npy", weights)


# Display feature importances (these are the learned "weights")
print("Feature Importances (Calculated Weights from Random Forest):")
for label, importance in zip(past_df.columns, feature_importances):
    print(f"{label}: {importance:.4f}")
# Step 2: PROMETHEE Method
import numpy as np

# Define the bipolar L-fuzzy covering values for each criterion (theta_i)
initial_matrix = np.array([
    [(0.6, -0.4), (0.2, -0.8), (0.4, -0.4), (0.8, -0.4)],  # Alt 1
    [(0.2, -0.8), (0.4, -0.6), (0.8, -0.4), (0.6, -0.2)],  # Alt 2
    [(0.8, -0.2), (0.6, -0.6), (0.4, -0.8), (0.4, -0.6)],  # Alt 3
    [(0.4, -0.2), (0.2, -0.8), (0.8, -0.2), (0.4, -0.4)]  # Alt 4
], dtype=object)

# Define G\"odel implication function for positive membership
def godel_implication_positive(a, b):
    return b if a > b else 1
 # G\"odel implication function for negative membership
def godel_implication_negative(a, b):
    return b if a > b else 0
   

# Initialize matrix to store combined bipolar entries as tuples (M^{(P)}, M^{(N)})
num_alternatives = initial_matrix.shape[0]
M_combined = np.empty((num_alternatives, num_alternatives), dtype=object)

# Calculate Positive L-neighborhood (M^{(P)}) and Negative L-neighborhood (M^{(N)})
for o in range(num_alternatives):
    for p in range(num_alternatives):
        positive_implications = [godel_implication_positive(initial_matrix[o, j][0], initial_matrix[p, j][0]) for j in range(initial_matrix.shape[1])]
        M_P = min(positive_implications)
        
        negative_tnorms = [min(initial_matrix[o, j][1], initial_matrix[p, j][1]) for j in range(initial_matrix.shape[1])]
        M_N = max(negative_tnorms)
        
        M_combined[o, p] = (round(M_P, 3), round(M_N, 3))

# Define eta and weights
eta = 0.4
w = 0.8
not_eta = 1 - eta
gamma = w * (-1 + eta) + (1 - w) * (1 - eta)
not_gamma = -1 - gamma

# Define S sets with positive and negative memberships
S_sets = {
    "S_1": {'o1': (0.6, -0.4), 'o2': (0.2, -0.8), 'o3': (0.8, -0.2), 'o4': (0.4, -0.2)},
    "S_2": {'o1': (0.2, -0.8), 'o2': (0.4, -0.6), 'o3': (0.6, -0.6), 'o4': (0.2, -0.8)},
    "S_3": {'o1': (0.4, -0.4), 'o2': (0.8, -0.4), 'o3': (0.4, -0.8), 'o4': (0.8, -0.2)},
    "S_4": {'o1': (0.8, -0.4), 'o2': (0.6, -0.2), 'o3': (0.4, -0.6), 'o4': (0.4, -0.4)}
}

# Initialize matrices for lower, upper approximations and the combined parameter matrix
lower_approx_matrix = np.zeros((num_alternatives, len(S_sets)), dtype=object)
upper_approx_matrix = np.zeros((num_alternatives, len(S_sets)), dtype=object)
parameter_matrix = np.zeros((num_alternatives, len(S_sets)), dtype=object)

# Functions to calculate lower and upper approximations
def lower_approximation_positive(o, S_set):
    index = int(o[1]) - 1
    return min([godel_implication_positive(M_combined[index][i][0], max(eta, S_set[f'o{i+1}'][0])) for i in range(len(M_combined[index]))])

def upper_approximation_positive(o, S_set):
    index = int(o[1]) - 1
    return max([min(M_combined[index][i][0], min(not_eta, S_set[f'o{i+1}'][0])) for i in range(len(M_combined[index]))])

def lower_approximation_negative(o, S_set):
    index = int(o[1]) - 1
    return max([min(M_combined[index][i][1], min(gamma, S_set[f'o{i+1}'][1])) for i in range(len(M_combined[index]))])

def upper_approximation_negative(o, S_set):
    index = int(o[1]) - 1
    return min([godel_implication_negative(M_combined[index][i][1], max(not_gamma, S_set[f'o{i+1}'][1])) for i in range(len(M_combined[index]))])

# Calculate lower, upper approximations and the parameter matrix for each S set
for col, (S_name, S_set) in enumerate(S_sets.items()):
    for row, o in enumerate(S_set.keys()):
        lower_pos = lower_approximation_positive(o, S_set)
        upper_pos = upper_approximation_positive(o, S_set)
        lower_neg = lower_approximation_negative(o, S_set)
        upper_neg = upper_approximation_negative(o, S_set)
        
        # Store results in lower and upper approximation matrices
        lower_approx_matrix[row, col] = (round(lower_pos, 3), round(lower_neg, 3))
        upper_approx_matrix[row, col] = (round(upper_pos, 3), round(upper_neg, 3))
        
        # Calculate the combined parameter matrix with vector addition
        parameter_matrix[row, col] = (lower_pos + upper_pos, lower_neg + upper_neg)

# Calculate the average normalized covering-based decision matrix
average_normalized_matrix = np.zeros((num_alternatives, len(S_sets)))

for row in range(num_alternatives):
    for col in range(len(S_sets)):
        p_ij_pos, p_ij_neg = parameter_matrix[row, col]
        average_normalized_matrix[row, col] = abs(p_ij_pos + p_ij_neg)

# Display the results
print("Lower Approximation Matrix ((Lower Positive, Lower Negative) for each S):")
for row in lower_approx_matrix:
    print([f"({entry[0]}, {entry[1]})" for entry in row])

print("\nUpper Approximation Matrix ((Upper Positive, Upper Negative) for each S):")
for row in upper_approx_matrix:
    print([f"({entry[0]}, {entry[1]})" for entry in row])

print("\nBipolar L-fuzzy Combined Parameter Matrix ((Combined Positive, Combined Negative) for each S):")
for row in parameter_matrix:
    print([f"({entry[0]}, {entry[1]})" for entry in row])

print("\nAverage Normalized Covering-Based Decision Matrix:")
print(average_normalized_matrix)
import numpy as np


# Weight vector for each criterion
weights = np.load("rf_weights.npy")

# Step 1: Define the preference function to set negative differences to zero
def preference_function(difference):
    return max(difference, 0)

# Step 2: Calculate the weighted preference matrix
num_alternatives, num_criteria = average_normalized_matrix.shape
preference_matrix = np.zeros((num_criteria, num_alternatives, num_alternatives))

for i in range(num_criteria):
    for j in range(num_alternatives):
        for k in range(num_alternatives):
            if j != k:
                # Calculate the difference between evaluations of alternatives for each criterion
                diff = average_normalized_matrix[j, i] - average_normalized_matrix[k, i]
                # Apply the preference function and multiply by the weight for that criterion
                preference_matrix[i, j, k] = preference_function(diff) * weights[i]

# Step 3: Calculate the multi-criteria preference index
preference_index = np.zeros((num_alternatives, num_alternatives))

for j in range(num_alternatives):
    for k in range(num_alternatives):
        if j != k:
            # Sum the weighted preferences across all criteria
            preference_index[j, k] = np.sum(preference_matrix[:, j, k])

# Step 4: Calculate positive and negative outranking flows
positive_flow = np.zeros(num_alternatives)
negative_flow = np.zeros(num_alternatives)

for j in range(num_alternatives):
    positive_flow[j] = np.sum(preference_index[j, :]) / (num_alternatives - 1)
    negative_flow[j] = np.sum(preference_index[:, j]) / (num_alternatives - 1)
   # Step 5: Calculate the net flow values and round to 3 decimal places
positive_flow = np.round(positive_flow, 3)
negative_flow = np.round(negative_flow, 3)
net_flow = np.round(positive_flow - negative_flow, 3)

# Step 6: Rank the alternatives based on net flow values
ranking = np.argsort(-net_flow)  # Sort in descending order (higher net flow is better)

# Display the results
print("Weighted Preference Matrix (P_j):")
print(preference_matrix)
print("\nMulti-criteria Preference Index (π):")
print(preference_index)
print("\nPositive Outranking Flow (ψ⁺):", positive_flow)
print("\nNegative Outranking Flow (ψ⁻):", negative_flow)
print("\nNet Flow (ψ):", net_flow)
for rank, alternative in enumerate(ranking, start=1):
    print(f"Rank {rank}: Alternative {alternative + 1} (Net Flow: {net_flow[alternative]:.3f})")


Feature Importances (Calculated Weights from Random Forest):
Energy Efficiency (Positive): 0.1943
Cycling Capability (Positive): 0.2851
Lifespan (Positive): 0.3581
Self-Discharge (Positive): 0.1625
Lower Approximation Matrix ((Lower Positive, Lower Negative) for each S):
['(0.6, -0.4)', '(0.4, -0.6)', '(0.4, -0.4)', '(0.8, -0.4)']
['(0.4, -0.4)', '(0.4, -0.6)', '(0.8, -0.4)', '(0.6, -0.36)']
['(0.8, -0.36)', '(0.6, -0.6)', '(0.4, -0.36)', '(0.4, -0.4)']
['(0.4, -0.36)', '(0.4, -0.6)', '(0.8, -0.36)', '(0.4, -0.4)']

Upper Approximation Matrix ((Upper Positive, Upper Negative) for each S):
['(0.6, -0.64)', '(0.4, -0.64)', '(0.4, -0.64)', '(0.6, -0.6)']
['(0.4, -0.64)', '(0.4, -0.64)', '(0.6, -0.64)', '(0.6, 0)']
['(0.6, -0.64)', '(0.6, -0.64)', '(0.4, -0.64)', '(0.4, -0.6)']
['(0.4, -0.64)', '(0.4, -0.64)', '(0.6, -0.64)', '(0.4, -0.6)']

Bipolar L-fuzzy Combined Parameter Matrix ((Combined Positive, Combined Negative) for each S):
['(1.2, -1.04)', '(0.8, -1.24)', '(0.8, -1.04)', '(1.4,